In [ ]:
# Imports
import websocket
import json
import pprint
import talib
import numpy as np
import config
import pandas as pd
import tkinter as tk
from talib import MA_Type
from binance.client import Client
from binance.client import *
from binance.enums import *
from threading import *
from tk import *
from websocket import *

In [ ]:
# Constants
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@miniTicker"
BOLLINGER_WINDOW = 10
BOLLINGER_WEIGHT = 1
WHALE_WEIGHT = 0
BUY_THRESHOLD = 1
SELL_THRESHOLD = -1
COOLDOWN_WINDOW = 3
BUY = 1
HOLD = 0
SELL = -1
TRADE_QUANTITY = 0.001
SYMBOL = 'BTCUSDT'
ORDER_TYPE = ORDER_TYPE_MARKET
MA_TYPE = 2

In [ ]:
# Global Variables
client = Client(config.API_KEY, config.API_SECRET, testnet = True)
financials_df = pd.DataFrame(columns = ['close', 'bollinger_signal', 'whale_signal', 'weighted_signal', 'trade_decision'])
cooldown_counter_buy = 0
cooldown_counter_sell = 0
initial_balance_btc = float(client.get_asset_balance(asset='BTC')['free'])
initial_balance_usdt = float(client.get_asset_balance(asset='USDT')['free'])
current_balance_btc = initial_balance_btc
current_balance_usdt = initial_balance_usdt

In [ ]:
# Creating Bollinger Signal
def get_bollinger_signal(historical_close):
    upper, middle, lower = talib.BBANDS(historical_close, timeperiod=BOLLINGER_WINDOW, matype=MA_TYPE)
    if historical_close[-1] < lower[-1]:
        bollinger_long = 1
    else:
        bollinger_long = 0
    if historical_close[-1] > upper[-1]:
        bollinger_short = -1
    else:
        bollinger_short = 0
    bollinger_signal = bollinger_long + bollinger_short
    return bollinger_signal

In [ ]:
# Creating Weighted Signal
def get_weighted_signal(signal_values, signal_weights):
    signal_weighted_avg = np.average(signal_values, weights=signal_weights)
    
    if signal_weighted_avg >= BUY_THRESHOLD:
        weighted_signal = BUY
    elif signal_weighted_avg <= SELL_THRESHOLD:
        weighted_signal = SELL
    else:
        weighted_signal = HOLD

    return weighted_signal

In [ ]:
# Creating Trade Decision
def get_trade_decision(current_signal):
    global cooldown_counter_buy
    global cooldown_counter_sell
    print('current signal is ', current_signal)
    print('buy cooldown counter is ')
    print(cooldown_counter_buy)
    print('sell cooldown counter is' )
    print(cooldown_counter_sell)
    trade_decision = 'hold'
    if current_signal == 1:
        if cooldown_counter_buy == 0:
            trade_decision = 'buy'
            cooldown_counter_buy = COOLDOWN_WINDOW
            cooldown_counter_sell = 0
            success, order_details = order('buy', TRADE_QUANTITY, SYMBOL, order_type=ORDER_TYPE)
            print("Success: ", success)
            print("Order Details: ", order_details)
            print(order_details)
        else:
            cooldown_counter_buy -= 1
    elif current_signal == -1:
        if cooldown_counter_sell == 0:
            trade_decision = 'sell'
            cooldown_counter_sell = COOLDOWN_WINDOW
            cooldown_counter_buy = 0
            success, order_details = order('sell', TRADE_QUANTITY, SYMBOL, order_type=ORDER_TYPE)
            print("Success: ", success)
            print("Order Details: ", order_details)
            print(order_details)
        else:
            cooldown_counter_sell -= 1
    else: #current_signal == 0:
        if cooldown_counter_buy > 0:
            cooldown_counter_buy -=1
        if cooldown_counter_sell > 0:
            cooldown_counter_sell -=1
            
    return trade_decision

In [ ]:
# Creating on_message
def on_message(ws, message):
    json_message = json.loads(message)
    close = json_message['c']
    financials_df_builder(float(close))
def on_open(ws):
    print('opened connection')
def on_close(ws):
    print('closed connection')

In [ ]:
# Creating Order
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
    try:
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
    except Exception as e:
        return False, e
    return True, order

In [ ]:
# Creating GUI

# Create an instance of the canvas
win = tk.Tk()

# Select the title of the window
win.title("BG-9000")

# Define the geometry of the window
win.geometry("600x650")

# Title
label_title = tk.Label(win, text="BITCOIN GENIE 9000",width=20,font=("bold", 48))
label_title.place(x=0,y=25)
    
# Labels / Text Boxes / Images
header_usdt= tk.Label(win, text="USDT Balance",width=20,font=("bold", 20))
header_usdt.place(x=295,y=120)

header_btc= tk.Label(win, text="BTC Balance",width=20,font=("bold", 20))
header_btc.place(x=20,y=120)

header_total_profit= tk.Label(win, text="Total Profit (USDT)",width=20,font=("bold", 20))
header_total_profit.place(x=155,y=220)

header_tradelog= tk.Label(win, text="Trade Log:",width=20,font=("bold", 20))
header_tradelog.place(x=-60,y=370)

label_disclaimer = tk.Label(win, text="DISCLAIMER: USE AT YOUR OWN PERIL!",width=60,font=("bold", 16))
label_disclaimer.place(x=0,y=600)

label_usdt= tk.Label(win,text= '%.2f' % initial_balance_usdt,font=("sans-serif", 36), fg= "green")
label_usdt.place(x=350,y=150)

label_btc= tk.Label(win,text= '%.3f' % initial_balance_btc, font=("sans-serf", 36), fg= "green")
label_btc.place(x=100,y=150)

label_total_profit= tk.Label(win,text= '0.0000', font=("sans-serf", 36), fg= "green")
label_total_profit.place(x=230,y=250)

text_tradelog= tk.Text(win, font=("bold", 16), height=10, width=55)
text_tradelog.place(x=20, y=400)

# Display function
def display():
    label_usdt.config(text= '%.2f' % financials_df['current_balance_usdt'].iloc[-1])
    label_btc.config(text= financials_df['current_balance_btc'].iloc[-1])
    label_total_profit.config(text= '%.4f' % (financials_df['delta_btc'].iloc[-1] * financials_df['close'].iloc[-1] + financials_df['delta_usdt'].iloc[-1]))
    secondsSinceEpoch = time.time()
    timeObj = time.localtime(secondsSinceEpoch)
    timestamp = ('%d-%d-%d %d:%d:%d' % (
    timeObj.tm_mday, timeObj.tm_mon, timeObj.tm_year, timeObj.tm_hour, timeObj.tm_min, timeObj.tm_sec))
    if financials_df['trade_decision'].iloc[-1] == 'buy':
        sentence = 'Bought ' + str(TRADE_QUANTITY) + " BTC at spot price of " + str(financials_df['close'].iloc[-1]) + " USDT"
        text_tradelog.insert(tk.END, '[' + timestamp + '] ' + sentence + '\n')
        text_tradelog.see('end')
    elif financials_df['trade_decision'].iloc[-1] == 'sell':
        sentence = "Sold " + str(TRADE_QUANTITY) + " BTC at spot price of " + str(financials_df['close'].iloc[-1]) + " USDT"
        text_tradelog.insert(tk.END, '[' + timestamp + '] ' + sentence + '\n')
        text_tradelog.see('end')
    win.update()

# Button function
def button_press():
    ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
    ws.run_forever()

# Button    
b1=tk.Button(win,text="ACTIVATE",font=("bold", 20),height=2,width=10,bg='blue',fg='black', command=button_press)
b1.place(x=210,y=300)

In [ ]:
# Creating financials_df_builder
def financials_df_builder(close):
    #this function's purpose is to append the latest information to the financials_df
    global financials_df
    global initial_balance_btc
    global initial_balance_usdt
    global current_balance_btc
    global current_balance_usdt
    signal_values = []
    signal_weights = []
    
    # Bollinger Signal and Weight
    bollinger_signal = 0
    if len(financials_df) >= BOLLINGER_WINDOW:
        bollinger_signal = get_bollinger_signal(financials_df['close'].values.astype(float))
    signal_values.append(bollinger_signal)
    signal_weights.append(BOLLINGER_WEIGHT)
    
    # Whale Signal and Weight
    whale_signal = 0
    signal_values.append(whale_signal)
    signal_weights.append(WHALE_WEIGHT)
    
    # Get overall weighted signal
    weighted_signal = get_weighted_signal(signal_values, signal_weights)
    
    # Trade Decision
    trade_decision = get_trade_decision(weighted_signal)
    
    # Currency Balances
    if trade_decision == 'buy' or trade_decision == 'sell':
        current_balance_btc = float(client.get_asset_balance(asset='BTC')['free'])
        current_balance_usdt = float(client.get_asset_balance(asset='USDT')['free'])
    
    # Delta BTC
    delta_balance_btc = current_balance_btc - initial_balance_btc
    
    # Delta USDT
    delta_balance_usdt = current_balance_usdt - initial_balance_usdt
    # Trade Log
    
    # Appending financials_df
    financials_df = financials_df.append({'close' : close, 'current_balance_btc' : current_balance_btc, 'delta_btc' : delta_balance_btc, 'current_balance_usdt' : current_balance_usdt, 'delta_usdt' : delta_balance_usdt, 'bollinger_signal' : bollinger_signal, 'whale_signal': whale_signal, 'weighted_signal' : weighted_signal, 'trade_decision' : trade_decision}, ignore_index = True)
    
    # Run display
    display()
    
    return

In [ ]:
# Keep Running the GUI Window
win.mainloop()